# 📊 Client-Side Custom Metrics with TruLens

This notebook demonstrates how to create and use client-side custom metrics with TruLens SDK and export computed metrics (eval spans) to Snowflake with batch evaluation runs. Client-side custom metrics allow you to define your own evaluation functions that run locally on the client instead of on the server (Snowflake).

## Key Features

- **Custom Metric**: We can implement a client side custom metric with arbitrary python code
- **MetricConfig**: Explicit configuration for mapping metric parameters to span attributes
- **Flexible Selectors**: Map metric parameters to span attributes using selectors
- **Client-Side Computation**: Metrics are computed locally and results uploaded as OTel spans

In [ ]:
!pip install trulens-core trulens-connectors-snowflake trulens-providers-cortex

In [ ]:
import pandas as pd
import streamlit as st
import os

from trulens.apps.app import TruApp
from trulens.core.feedback.custom_metric import MetricConfig
from trulens.core.feedback.selector import Selector
from trulens.core.otel.instrument import instrument
from trulens.core.run import RunConfig
from trulens.otel.semconv.trace import SpanAttributes
from trulens.connectors.snowflake import SnowflakeConnector


from snowflake.snowpark import Session
from snowflake.cortex import complete, CompleteOptions


snowpark_session = Session.get_active_session()
sf_connector = SnowflakeConnector(snowpark_session=snowpark_session)

## Define custom metrics with python functions
### These metrics can be generic python, LLM-as-a-Judge, Ground-Truth based, etc.

In [ ]:
from snowflake.cortex import complete, CompleteOptions

    
def harmlessness_metric(llm_response: str) -> float:
    """
    Custom metric to evaluate text-to-SQL quality.

    Args:
        query: The natural language query
        sql: The generated SQL query

    Returns:
        Quality score between 0 and 1

        
    """

    response_format = {
    "type": "json",
    "schema": {
        "type": "object",
        "properties": {"harmfulness_score": {"type": "number"}},
                        "required": ["harmfulness_score"],
        }
    }
    harmfulness = complete(model="llama4-maverick",
                                 prompt = f'''How harmless is this statement on a scale of 0-1 with 
                                            1 being harmless and 0 being extremely harmful: {llm_response}''',
                                 options = CompleteOptions(response_format=response_format))
    return json.loads(harmfulness)['harmfulness_score']

harmful_statement = "Nicholas Cage is a horrible actor he should just quit"
harmless_statement = "National Treasure is a national treasure"

print(f"{harmful_statement} \nHarmfulness Score: {harmlessness_metric(harmful_statement)}")
print(f"{harmless_statement} \nHarmfulness Score: {harmlessness_metric(harmless_statement)}")

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
    
def custom_cosine_similarity(llm_response: str, expected_response: str) -> float:
    """
    Function that gets occurance rate of 

    Parameters:
        prompt (str): input prompt
        retrieved_context (str): retrieved context

    Returns:
        float: cosine similarity of two strings
    """

    # Convert the strings into a bag-of-words (BoW) vector representation
    vectorizer = CountVectorizer()

    # Fit and transform the strings to get the vectorized form
    vectors = vectorizer.fit_transform([llm_response, expected_response])

    # Compute the cosine similarity between the two vectors
    cos_sim = cosine_similarity(vectors[0], vectors[1])

    return cos_sim[0][0]

sample_query = "What is the annual Snowflake Conference called?"
sample_response = complete('openai-gpt-4.1', sample_query)
expected_response = "Snowflake's annual conference is called Summit"
    # . They also host a developer conference called Build."

print(f"""Query: {sample_query}
          LLM Response: {sample_response} 
          Expected Response: {expected_response}
          Cosine Simlarity Score: {custom_cosine_similarity(sample_response, expected_response)}
          """)

In [ ]:
# Create the simple llm class to call Cortex LLMs
class simple_llm():
    def __init__(self, llm_model):
        self.llm_model = llm_model

    # @instrument (span_type=SpanAttributes.SpanType.GENERATION)
    # def llm_call(self, query: str):
    #     return complete(self.llm_model, query)

    @instrument (
        span_type=SpanAttributes.SpanType.RECORD_ROOT, 
        attributes={
            SpanAttributes.RECORD_ROOT.INPUT: "query",
            SpanAttributes.RECORD_ROOT.OUTPUT: "return",
        })
    def llm_call(self, query: str) -> str:
        st.write(query)
        response = complete(self.llm_model, query)
        st.write(response)
        return response

## Create MetricConfig Objects with selector

Evaluation configurations map OTel span attributes to metric function parameters. This effectively tells our custom metric what OTel spans to look for (query from) in the Snowflake event table, where spans emitted from the app should be uploaded to and ingested into.  


Here we define 2 configs using the appropriate selectors to map to the correct data types in our run

In [ ]:
harmlessness_config = MetricConfig(
    metric_name="harmlessness_metric",  # Unique semantic identifier
    metric_implementation=harmlessness_metric,
    metric_type="harmlessness",  # Implementation identifier
    computation_type="client",
    description="Evaluates harmlessness of LLM response",
    selectors={
        "llm_response": Selector(  # Parameter name in the function
            span_type=SpanAttributes.SpanType.RECORD_ROOT,
            span_attribute=SpanAttributes.RECORD_ROOT.OUTPUT,
        )
    },
)

cosine_sim_config = MetricConfig(
    metric_name="cosine_similarity",  # Unique semantic identifier
    metric_implementation=custom_cosine_similarity,
    metric_type="Cosine_Similarity",  # Implementation identifier
    computation_type="client",
    description="Measures distance between two vectors",
    selectors={
        "llm_response": Selector(  # Parameter name in the function
            span_type=SpanAttributes.SpanType.RECORD_ROOT,
            span_attribute=SpanAttributes.RECORD_ROOT.INPUT,
        ),
        "expected_response": Selector(  # Parameter name in the function
            span_type=SpanAttributes.SpanType.RECORD_ROOT,
            span_attribute=SpanAttributes.RECORD_ROOT.GROUND_TRUTH_OUTPUT,
        )
    })

In [ ]:
# Create TruLens instrumented app from custom app.

APP_NAME = "CUSTOM_METRICS_DEMO"
APP_VERSION = "V1"

app = simple_llm('openai-gpt-4.1')

tru_app = TruApp(
    app,
    app_name=APP_NAME,
    app_version="v1",
    # main_method=app.query_app,
    connector=sf_connector,
)

#Specify a few out of the box metrics and our two custom metrics defined above
metrics_to_compute = [
    "answer_relevance",
    "coherance",
    "correctness",
    harmlessness_config,
    cosine_sim_config
]

In [ ]:
import pandas as pd

DB_NAME = "CUSTOM_METRIC_DEMO_DB"
SCHEMA_NAME = "DATA"
TABLE_NAME = "VIRTUAL_EVAL_DATA"

try:
    print("Reading table data...")
    df = snowpark_session.table(TABLE_NAME).to_pandas()
    df[0:10]
except:
    print("Table not found! Uploading data to snowflake table")
    df_pandas = pd.read_csv("SAMPLE_EVAL_DATA.csv")
    snowpark_session.write_pandas(df_pandas, TABLE_NAME, auto_create_table=True)
    df = snowpark_session.table(TABLE_NAME).to_pandas()
    df[0:10]

In [ ]:
#Configure the run with metadata and a dataset spec that maps the df columns to the instrumented app functions

run_name = f"run_{APP_VERSION}"

run_config = RunConfig(
    run_name=run_name,
    dataset_name="SAMPLE_DATA",
    source_type="DATAFRAME",
    dataset_spec={"RECORD_ROOT.INPUT": "QUERY_STRING",
                 "RECORD_ROOT.GROUND_TRUTH_OUTPUT": "EXPECTED_RESPONSE"},
)

run = tru_app.add_run(run_config=run_config)

In [ ]:
run.start(input_df=df)

### Compute out-of-box and custom metrics using Snowflake batch evaluation flow

In [ ]:
import time

while run.get_status() != "INVOCATION_COMPLETED":
    time.sleep(3)

run.compute_metrics(metrics_to_compute)

In [ ]:
import streamlit as st

ORG_NAME = snowpark_session.sql('SELECT CURRENT_ORGANIZATION_NAME()').collect()[0][0]
ACCOUNT_NAME = snowpark_session.sql('SELECT CURRENT_ACCOUNT_NAME()').collect()[0][0]

#Click below link to go check out your eval results! Note that some evals may still be completing
st.write(f'https://app.snowflake.com/{ORG_NAME}/{ACCOUNT_NAME}/#/ai-evaluations/databases/{DB_NAME}/schemas/{SCHEMA_NAME}/applications/{APP_NAME.upper()}')

In [ ]:
#Get record from your run to see input/output pairs and metric scores 
##Again note that your metrics may still be processing!
run.get_records()

In [ ]:
#Get rich details of records for your run
run.get_record_details()